# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient({"localhost:27017"})
db=db = client.get_database("ironhack")

c=db.get_collection("companies ")
db.list_collection_names()

['companies ']

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
query = {'name' : 'Babelgum'}
proyect = {'name' : 1, '_id':0}
list(c.find(query,proyect).limit(10))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [3]:
query={"number_of_employees":{"$gt":500}}
list(c.find(query).sort("number_of_employees",-1).limit(20))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'permalink': 'siemens',
  'crunchbase_url': 'http://www.crunchbase.com/company/siemens',
  'homepage_url': 'http://www.siemens.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Siemens',
  'category_code': 'hardware',
  'number_of_employees': 405000,
  'founded_year': 1847,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'automation, building-technologies, drive-technology, energy',
  'alias_list': '',
  'email_address': 'contact@siemens.com',
  'phone_number': '49 89 636 34134',
  'description': 'Electronics and Electrical Engineering',
  'created_at': 'Thu Jul 31 09:29:43 UTC 2008',
  'updated_at': 'Thu Nov 28 20:32:55 UTC 2013',
  'overview': '<p>Siemens AG, an electronics and electrical engineering company, operates in the industry, energy, and healthcare sectors worldwide. 

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [4]:

query={"$and":[{"founded_year":{"$lte":2005}},{"founded_year":{"$gte":2000}}]}
proyect = {'name' : 1, "founded_year": 1, '_id':0}
list(c.find(query,proyect).limit(20))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [5]:
query={'ipo.valuation_amount':{"$gt":100000000}}
query2={"founded_year":{"$lt":2010}}
and_={"$and":[query,query2]}
proyect = {'name' : 1, 'ipo': 1, '_id':0}
list(c.find(and_,proyect).limit(10)) #limito a diez para no inundar

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Brightcove',
  'ipo': {

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [6]:
query={"number_of_employees":{"$lt":1000}}
query2={"founded_year":{"$lt":2005}}
and_={"$and":[query,query2]}
proyect = {'name' : 1, '_id':0}
list(c.find(and_,proyect).sort("number_of_employees",1).limit(10)) 

[{'name': 'Skype'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Ticketmaster'},
 {'name': 'Fox Interactive Media'},
 {'name': 'stylediary'},
 {'name': 'Monster'},
 {'name': 'EditGrid'},
 {'name': 'Compete'},
 {'name': 'MindTouch'}]

### 6. All the companies that don't include the `partners` field.

In [7]:
query= {'partners' : {"$exists": False}}
list(c.find(query,).sort("number_of_employees",1).limit(10)) 

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [8]:
query= {'category_code' : None}
list(c.find(query).sort("number_of_employees",1).limit(10)) #limito a diez para no inundar

[{'_id': ObjectId('52cdef7c4bab8bd67529827a'),
  'name': 'MMDAYS',
  'permalink': 'mmdays',
  'crunchbase_url': 'http://www.crunchbase.com/company/mmdays',
  'homepage_url': None,
  'blog_url': None,
  'blog_feed_url': None,
  'twitter_username': None,
  'category_code': None,
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': None,
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Tue Feb 12 08:00:42 UTC 2008',
  'updated_at': 'Fri Feb 15 05:36:52 UTC 2008',
  'overview': None,
  'image': None,
  'products': [{'name': 'KusoPK', 'permalink': 'kusopk'}],
  'relationships': [],
  'competitions': [],
  'providerships': [],
  'total_money_raised': '$0',
  'funding_rounds': [],
  'investments': [],
  'acquisition': None,
  'acquisitions': [],
  'offices': [],
 

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [9]:
query={"$and": [{'number_of_employees':{"$gte":100}},{'number_of_employees': {"$lt":1000}}]}
proyect = {'name' : 1, 'number_of_employees' : 1, '_id' :0}
list(c.find(query,proyect).limit(10)) #limito a diez para no inundar

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [10]:

project = {"name":1, 'ipo': 1, '_id' :0} #pongo ipo para poder comprobar el nombre
list(c.find({},project).sort("ipo", -1).limit(10)) #limito 

[{'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Groupon',
  'ipo': {'valuation_amount': 12800000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NASDAQ:GRPN'}},
 {'name': 'Tencent',
  'ipo': {'v

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [11]:
proyect = {'name' : 1, 'number_of_employees' : 1, '_id' :0} #pongo solo nombre y num de empleados para comprobar el orden
list(c.find({}).sort("number_of_employees",-1).limit(10))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'permalink': 'siemens',
  'crunchbase_url': 'http://www.crunchbase.com/company/siemens',
  'homepage_url': 'http://www.siemens.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Siemens',
  'category_code': 'hardware',
  'number_of_employees': 405000,
  'founded_year': 1847,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'automation, building-technologies, drive-technology, energy',
  'alias_list': '',
  'email_address': 'contact@siemens.com',
  'phone_number': '49 89 636 34134',
  'description': 'Electronics and Electrical Engineering',
  'created_at': 'Thu Jul 31 09:29:43 UTC 2008',
  'updated_at': 'Thu Nov 28 20:32:55 UTC 2013',
  'overview': '<p>Siemens AG, an electronics and electrical engineering company, operates in the industry, energy, and healthcare sectors worldwide. 

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [12]:
query={"founded_month":{"$gt":6}}
proyect = {'name' : 1, 'created_at' : 1, '_id' :0}
list(c.find({},proyect).limit(1000))

[{'name': 'Wetpaint', 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27)},
 {'name': 'AdventNet',
  'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22)},
 {'name': 'Zoho', 'created_at': 'Fri May 25 19:30:28 UTC 2007'},
 {'name': 'Digg', 'created_at': 'Fri May 25 20:03:23 UTC 2007'},
 {'name': 'Facebook', 'created_at': 'Fri May 25 21:22:15 UTC 2007'},
 {'name': 'Omnidrive', 'created_at': 'Sun May 27 03:25:32 UTC 2007'},
 {'name': 'Postini', 'created_at': 'Fri Jun 08 12:19:51 UTC 2007'},
 {'name': 'Geni', 'created_at': 'Thu May 31 19:52:34 UTC 2007'},
 {'name': 'Flektor', 'created_at': 'Thu May 31 21:11:51 UTC 2007'},
 {'name': 'Fox Interactive Media',
  'created_at': 'Thu May 31 21:46:57 UTC 2007'},
 {'name': 'Twitter', 'created_at': 'Fri Jun 01 08:42:34 UTC 2007'},
 {'name': 'StumbleUpon',
  'created_at': datetime.datetime(2007, 6, 1, 9, 28, 13)},
 {'name': 'Gizmoz', 'created_at': 'Fri Jun 01 19:32:24 UTC 2007'},
 {'name': 'Scribd', 'created_at': 'Sat Jun 02 07:34:05 UTC 2007

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [13]:
query={"$and": [{'founded_year':{"$lt":2000}},{'acquisition.price_amount': {"$gt":10000000}}]}

list(c.find(query).limit(10)) #limito para no inundar

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'permalink': 'postini',
  'crunchbase_url': 'http://www.crunchbase.com/company/postini',
  'homepage_url': 'http://postini.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': None,
  'category_code': 'web',
  'number_of_employees': None,
  'founded_year': 1999,
  'founded_month': 6,
  'founded_day': 2,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': '',
  'alias_list': None,
  'email_address': '',
  'phone_number': '888.584.3150',
  'description': None,
  'created_at': 'Fri Jun 08 12:19:51 UTC 2007',
  'updated_at': 'Sat Aug 13 18:02:34 UTC 2011',
  'overview': '<p>Postini focuses on two main issues: security and compliance. Postini states that it handles more than 1 billion messages everyday and protects more than 35,000 businesses worldwide.</p>\n\n<p>Postini offers solutions that protect your company from malicious internet a

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [14]:
query={'acquisition.acquired_year': {"$gt":2010}}
proyect = {'name' : 1, "acquisition.price_amount":1 , '_id' :0}
list(c.find(query,proyect).sort("acquisition.price_amount",-1).limit(10)) #limito para o inundar

[{'name': 'T-Mobile', 'acquisition': {'price_amount': 39000000000}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000}},
 {'name': 'LSI', 'acquisition': {'price_amount': 6600000000}},
 {'name': 'National Semiconductor',
  'acquisition': {'price_amount': 6500000000}},
 {'name': 'Ariba', 'acquisition': {'price_amount': 4300000000}},
 {'name': 'NetLogic Microsystems',
  'acquisition': {'price_amount': 3700000000}},
 {'name': 'SuccessFactors', 'acquisition': {'price_amount': 3400000000}},
 {'name': 'Qualcomm Atheros', 'acquisition': {'price_amount': 3100000000}},
 {'name': 'Global Crossing', 'acquisition': {'price_amount': 3000000000}},
 {'name': 'Global Crossing', 'acquisition': {'price_amount': 3000000000}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [15]:
proyect = {'name' : 1, "founded_year":1 , '_id' :0}
list(c.find({},proyect).sort("founded_year",-1).limit(10)) #limito par ano inundar

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [16]:
query={"founded_day":{"$lte":7}}
proyect = {'name' : 1, 'founded_day' : 1, '_id' :0}
list(c.find({},proyect).sort("acquisition.price_amount",1).limit(10))

[{'name': 'AdventNet'},
 {'name': 'Fox Interactive Media', 'founded_day': 1},
 {'name': 'Gizmoz', 'founded_day': 1},
 {'name': 'Geni', 'founded_day': 1},
 {'name': 'Omnidrive', 'founded_day': 1},
 {'name': 'Twitter', 'founded_day': 21},
 {'name': 'Slacker', 'founded_day': None},
 {'name': 'Scribd', 'founded_day': None},
 {'name': 'Facebook', 'founded_day': 1},
 {'name': 'Zoho', 'founded_day': 15}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [17]:
query={"$and":[{"number_of_employees":{"$gt":4000}},{"category_code":"web"}]}
list(c.find(query).sort("number_of_employees",1).limit(10)) #limito a 10

[{'_id': ObjectId('52cdef7c4bab8bd67529822a'),
  'name': 'Expedia',
  'permalink': 'expedia',
  'crunchbase_url': 'http://www.crunchbase.com/company/expedia',
  'homepage_url': 'http://www.expedia.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Expedia',
  'category_code': 'web',
  'number_of_employees': 4400,
  'founded_year': 1996,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'travel, flights, hotels, tickets, airline, vacation, travel-comparison, travel-search',
  'alias_list': '',
  'email_address': '',
  'phone_number': '1-800-EXPEDIA',
  'description': '',
  'created_at': 'Sun Feb 10 03:29:12 UTC 2008',
  'updated_at': 'Sun Dec 08 07:03:33 UTC 2013',
  'overview': '<p>Expedia.com is an Internet-based travel agency and a part of Expedia, Inc.. It books airline tickets, hotel reservations, car rentals, cruises, vacation packages, and vari

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [18]:
query={"$and":[{"acquisition.price_amount":{"$gt":10000000}},{"acquisition.price_currency_code":"EUR"}]}
list(c.find(query).sort("number_of,_employees",1).limit(10)) #limito a 10 pra no inundar

[{'_id': ObjectId('52cdef7c4bab8bd675297f02'),
  'name': 'ZYB',
  'permalink': 'zyb',
  'crunchbase_url': 'http://www.crunchbase.com/company/zyb',
  'homepage_url': 'http://zyb.com',
  'blog_url': 'http://blog.zyb.com/',
  'blog_feed_url': 'http://blog.zyb.com/index.php/feed/',
  'twitter_username': None,
  'category_code': 'mobile',
  'number_of_employees': 25,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'mobile, social-networking, socialnetwork',
  'alias_list': '',
  'email_address': 'contact@zyb.com',
  'phone_number': '',
  'description': 'Mobile Social Networking',
  'created_at': 'Wed Aug 22 05:14:51 UTC 2007',
  'updated_at': 'Tue Jul 02 03:00:50 UTC 2013',
  'overview': '<p>ZYB is a mobile phone utility and social networking site. ZYB started in 2005 as a european service to provide easy back-up services for mobile phones. Using t

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [19]:
query={"acquisition.acquired_month":{"$lte":3}}
project={"name":1, "acquisition":1, "_id":0}
list(c.find(query,project).limit(10)) #limito a 10


[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [20]:
c1={"founded_year":{"$gte":2000}}
c2={"founded_year":{"$lte":2010}}
c3={"acquisition.acquired_year":{"$gte":2011}}
query={"$and":[c1,c2,c3]}
list(c.find(query).limit(10))#limito para no inundar

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 20. All the companies that have been 'deadpooled' after the third year.

In [23]:
#he estado buscand pero lo doy por imposible.
list(c.find( { "$where" : "this.founded_year - this.deadpooled_year> 3" } ).limit(10))#limito para no inundar

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon